In [1]:
import ollama
import pandas as pd
from datasets import load_dataset

## SQuAD V1

In [2]:
dataset = load_dataset("squad")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
df = pd.DataFrame(dataset['train']).sample(n=10)

In [41]:
validationDf = pd.DataFrame(dataset['validation']).sample(n=1000)

In [50]:
response = ollama.chat(model="mistral", messages=[{
    'role': 'user',
    'content': 'I am performing extractive question answering. I will provide you a context, question and answer. You need to provide me if the answer is correct or wrong. Just write "Correct" or "Wrong"'
}])

In [10]:
response['message']['content']

' Understood. I will judge whether the provided answer is correct or wrong based on the given context and question.\n\nContext: Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services. Its hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple Watch smartwatch, the AirPods wireless earbuds, and the Apple TV digital media player.\n\nQuestion: Which company manufactures the iPad?\nAnswer: Microsoft Corporation.\n\nVerdict: Wrong.'

In [14]:
response = ollama.chat(model="mistral", messages=[{
    'role': 'user',
    'content': 'Just provide me "Correct" or "Wrong". Don\'t provide anything else'
}])

In [15]:
response['message']['content']

' I. The capital city of France is:\n\nA) London\nB) Madrid\nC) Berlin\nD) Paris\n\nAnswer: D. Paris.\n\nII. The largest planet in our solar system is:\n\nA) Mars\nB) Jupiter\nC) Venus\nD) Saturn\n\nAnswer: B. Jupiter.\n\nIII. The currency of Japan is:\n\nA) Euro\nB) Pound\nC) Yen\nD) Dollar\n\nAnswer: C. Yen.\n\nIV. The tallest mountain in the world is:\n\nA) Everest\nB) Kilimanjaro\nC) K2\nD) Mauna Loa\n\nAnswer: A. Mount Everest.\n\nV. The largest country by land area is:\n\nA) Russia\nB) Canada\nC) United States\nD) China\n\nAnswer: A. Russia.\n\nVI. The human body contains approximately:\n\nA) 78 organs\nB) 100 trillion cells\nC) 5 liters of blood\nD) 206 bones\n\nAnswer: B. 100 trillion cells.\n\nVII. The deepest part of the ocean is:\n\nA) Mariana Trench\nB) Red Sea\nC) Pacific Ocean\nD) Indian Ocean\n\nAnswer: A. Mariana Trench.'

In [20]:
response = ollama.chat(model="mistral", messages=[{
    'role': 'user',
    'content': 'Why all the answers you are providing are having an explaination.Just provide me "Correct" or "Wrong". Don\'t provide anything else'
}])

In [21]:
response['message']['content']

' I apologize for any previous responses that included explanations when you only requested a "Correct" or "Wrong." I will make sure to keep my answers brief going forward. However, please note that providing a simple "Correct" or "Wrong" may not always be sufficient, as it depends on the context and complexity of the question. If you have a specific factual query, feel free to ask and I\'ll do my best to give you a succinct answer.'

In [34]:
def getModelFeedback(row, model):
    context = row["context"]
    question = row["question"]
    answer = row["answers"]
    
    prompt = f"I am providing you the context, question and answer. Please tell me if it is correct or not.Just provide 'correct' or 'wrong'. Do not provide any explaination or anything else.\nContext: {context}\nQuestion: {question}\nAnswer: {answer}"
    
    response = ollama.chat(model=model, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    
    return response['message']['content']
    

In [26]:
df["Mistral Verdict"] = df.apply(getModelFeedback, model="mistral", axis=1)

In [27]:
df

,id,title,context,question,answers,Mistral Verdict
4306,56cfad77234ae51400d9be5e,New_York_City,More than 200 newspapers and 350 consumer maga...,Which New York-based newspaper has won the Pul...,"{'text': ['The New York Times'], 'answer_start...","correct. However, it's important to note that..."
43587,5726bea0708984140094d01e,Mexico_City,The Centro Nacional de las Artes (National Cen...,What is the name of the CCU center opened in 2...,"{'text': ['Tlatelolco'], 'answer_start': [622]}",correct.
39573,572802ae4b864d1900164215,Northwestern_University,Many students are involved in community servic...,What is the name of the university's group ser...,{'text': ['Global Engagement Summer Institute'...,correct.
74522,572ee21dc246551400ce476e,Transistor,The essential usefulness of a transistor comes...,What is an additional use of the transistor?,{'text': ['turn current on or off in a circuit...,correct.\n\nThe context explains that a trans...
45773,5726e0c55951b619008f8123,Predation,Mimicry is a related phenomenon where an organ...,What is the phenomenon where an organism looks...,"{'text': ['Mimicry'], 'answer_start': [0]}",correct.
74564,572f9165a23a5019007fc774,Transistor,FETs are further divided into depletion-mode a...,What channel corresponds with high current?,"{'text': ['n-channel devices'], 'answer_start'...",Correct. The context states that a more posit...
46013,57266a9bf1498d1400e8df16,British_Empire,"At the concluding Treaty of Utrecht, Philip re...",Which country did Britain acquire Gibraltar an...,"{'text': ['Spain'], 'answer_start': [252]}",correct.
25317,5706e5579e06ca38007e91fe,"Atlantic_City,_New_Jersey","Marvin Gardens, the leading yellow property on...",In what year did Parker Brothers acknowledge a...,"{'text': ['1995'], 'answer_start': [326]}",correct.
21641,56f970b39e9bad19000a091c,List_of_numbered_streets_in_Manhattan,181st Street is a major thoroughfare running t...,Which river does 181st Street run near?,"{'text': ['Hudson River'], 'answer_start': [221]}",correct.
2256,56ccf12b62d2951400fa64f4,Sino-Tibetan_relations_during_the_Ming_dynasty,The Columbia Encyclopedia distinguishes betwee...,What did Thomas Laird dismiss the Yuan dynasty...,"{'text': ['a non-Chinese polity'], 'answer_sta...",correct.


In [42]:
validationDf["Mistral Verdict"] = validationDf.apply(getModelFeedback, model="mistral", axis=1)

In [43]:
validationDf

,id,title,context,question,answers,Mistral Verdict
763,56d9ca0adc89441400fdb821,Super_Bowl_50,There would be no more scoring in the third qu...,What yard line was the Broncos on when Manning...,"{'text': ['50-yard line.', '41', '50'], 'answe...",wrong. The context states that Ealy recovered...
5859,5727403af1498d1400e8f527,American_Broadcasting_Company,At the same time he made attempts to help grow...,What Western was a flagship program for ABC ar...,"{'text': ['The Lone Ranger', 'The Lone Ranger'...",correct.
8575,57292994af94a219006aa131,Kenya,"In the motor rallying arena, Kenya is home to ...",What is Kenya the home of?,"{'text': ['the world famous Safari Rally', 'Sa...",correct.
5450,57269d68708984140094cbd8,Victoria_and_Albert_Museum,The interiors of the three refreshment rooms w...,Who designed the ceiling and stained-glass win...,"{'text': ['Edward Burne-Jones', 'Edward Burne-...",Correct.
6958,57274e975951b619008f87fa,Construction,Several project structures can assist the owne...,These project structures allow the owner to in...,"{'text': ['architects, interior designers, eng...","correct.\n\nThe context mentions ""architects,..."
...,...,...,...,...,...,...
6882,5728ed94ff5b5019007da97f,Civil_disobedience,"Howard Zinn writes, ""There may be many times w...",Why should one not go to jail?,"{'text': ['accept jail penitently', 'is to swi...",Wrong. The context states that one should not...
2826,57094d489928a8140047150d,Sky_(United_Kingdom),BSkyB utilises the VideoGuard pay-TV scramblin...,Who has design authority over all of the digit...,"{'text': ['BSkyB', 'BSkyB', 'BSkyB'], 'answer_...",correct.
9388,573007fab2c2fd140056876f,Rhine,From the death of Augustus in AD 14 until afte...,Which direction did Romans use to drift throug...,"{'text': ['eastwards', 'eastwards', 'eastwards...",correct.
8353,5728dab94b864d1900164f98,Kenya,"Kenya (/ˈkɛnjə/; locally [ˈkɛɲa] ( listen)), o...",What is the capitol of Kenya?,"{'text': ['Nairobi', 'Nairobi', 'Nairobi'], 'a...",Correct.


In [44]:
validationDf.to_csv("squad_correct_wrong_prompt_mistral.csv")

## SQUAD V2

In [45]:
squadV2 = load_dataset("rajpurkar/squad_v2")

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [46]:
v2Validation = pd.DataFrame(squadV2['validation']).sample(n=100)

In [47]:
v2Validation["mistral Verdict"] = v2Validation.apply(getModelFeedback, model="mistral", axis=1)

In [48]:
v2Validation

,id,title,context,question,answers,mistral Verdict
3205,5ad14966645df0001a2d1583,European_Union_law,None of the original treaties establishing the...,What other entity was not established at the s...,"{'text': [], 'answer_start': []}",wrong. The European Court of Human Rights was...
9073,5ad268a9d7d075001a429281,Rhine,The mouth of the Rhine into Lake Constance for...,What is one of the mammal animals that lives i...,"{'text': [], 'answer_start': []}",Wrong. The question asks for a mammal animal ...
10074,5acfebe477cf76001a6864d8,Islamism,These attacks resonated with conservative Musl...,What did Saudi Arabia not try to repress to co...,"{'text': [], 'answer_start': []}",Wrong. The question asks about what Saudi Ara...
9154,572f64ccb2c2fd14005680bb,Rhine,The Upper Rhine region was changed significant...,What is the Bassin de compensation de Plobshei...,"{'text': ['large compensation pools', 'large c...",correct.
119,5ad3f7ac604f3c001a3ffa3d,Normans,One of the claimants of the English throne opp...,Who did the Scotish king take hostage?,"{'text': [], 'answer_start': []}",Wrong. The Scottish king took the Scottish pr...
...,...,...,...,...,...,...
2265,5711669550c2381900b54adf,Steam_engine,The Rankine cycle is sometimes referred to as ...,What is the Rankine cycle sometimes called?,"{'text': ['practical Carnot cycle', 'practical...",correct.
9089,5ad26aa8d7d075001a429324,Rhine,"A regulation of the Rhine was called for, with...",What lake no longer has any silt?,"{'text': [], 'answer_start': []}",Wrong. The context states that it is expected...
3360,5726c5a9f1498d1400e8eac8,European_Union_law,"In regard to companies, the Court of Justice h...",In which case did the Court of Justice hold th...,{'text': ['Überseering BV v Nordic Constructio...,Correct.
8543,57293e983f37b3190047818b,Intergovernmental_Panel_on_Climate_Change,"In 2001, 16 national science academies issued ...",When was the joint statement on climate change...,"{'text': ['2001', '2001', '2001'], 'answer_sta...",Correct.


## AdversarialQA 

In [52]:
adversarialData = load_dataset("UCLNLP/adversarial_qa", "adversarialQA")

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [53]:
adversarialData

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'metadata'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'metadata'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'metadata'],
        num_rows: 3000
    })
})

In [57]:
adversarialValidation = adversarialData["validation"]

In [76]:
adversarialValidation[20]

{'id': 'ba0946490bf8d1228e12756d6ecc7d9895f6a693',
 'title': 'Newcastle_upon_Tyne',
 'context': 'Following guidelines set in the National Cycling strategy, Newcastle first developed its cycling strategy in 1998. As of 2012, the local council social aims and objectives for cycling include: highlighting the usage of cycling to cut city congestion; educating that cycling promotes healthy living… The authority also has infrastructure aims and objectives which include: developing on road cycle networks on quieter streets; making safer routes on busier streets; innovating and implementing contraflows on one way streets; developing the existing off road cycle route networks and improve signage; joining up routes that are partially or completely isolated; Increase the number of cycle parking facilities; working with employers to integrate cycling into workplace travel plans; link the local networks to national networks.',
 'question': 'What has been added to streets to allow traffic in both di

In [75]:
getModelFeedback(adversarialValidation[20], model='mistral')

' correct.'